In [ ]:
from scripts.utils import get_device
from scripts.train_model import train_model
from scripts.test_model import test_model
from scripts.utils import convert_to_color_

import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.metrics import classification_report as score
from scipy.io import loadmat

import torch

## Проверяем доступность GPU для вычислений

In [ ]:
torch.cuda.is_available()

## Задаем параметры запуска


In [ ]:
DATASET_PATH: str = 'data/med/' # путь до папки с данными
IMG_NAME: str = 'iz3kubov.mat' # название .mat файла с гиперспектральным изображением (ключ по умолчанию 'image')
GT_NAME: str = 'iz3kubov_gt.mat' # название .mat файла с маской (ключ по умолчанию 'img')
WEIGHTS_PATH: str = 'checkpoints/short_he/he/2022_05_05_14_04_27_epoch15_0.96.pth' # путь до файла с весами (опционально)
SAMPLE_PERCENTAGE: float = 0.1 # размер тренировочной выборки из куба
CUDA_DEVICE = get_device(0) # подключение к доступному GPU, иначе подключается CPU

## Задаем гиперпараметры для сети

In [ ]:
# Указываем количество эпох, классов и устройство для вычисления
hyperparams = {
        'epoch': 15,
        'device': CUDA_DEVICE
    }

## Вызов обучения сети

In [ ]:
train_model(dataset_path=DATASET_PATH,
                img_name=IMG_NAME,
                gt_name=GT_NAME,
                sample_percentage=SAMPLE_PERCENTAGE,
                hyperparams=hyperparams)

## Вызов предсказания сети

In [ ]:
gt, predict, predict_color = test_model(dataset_path=DATASET_PATH,
                                img_name=IMG_NAME,
                                gt_name=GT_NAME,
                                hyperparams=hyperparams,
                                weights_path=WEIGHTS_PATH
                            )

### Задаем палитру для отрисовки результатов предсказания

In [ ]:
palette = {0: (0, 0, 0)}
for k, color in enumerate(sns.color_palette("hls", len(LABEL_VALUES) - 1)):
    palette[k + 1] = tuple(np.asarray(255 * np.array(color), dtype="uint8"))

### Отрисовываем результаты

In [ ]:
plt.figure(figsize=(5,5))
plt.imshow(convert_to_color_(gt, palette=palette))

plt.figure(figsize=(5,5))
plt.imshow(predict_color)

img = loadmat(f'{DATASET_PATH}/{IMG_NAME}')['image']
plt.figure(figsize=(5,5))
plt.imshow(img[:,:,100])

In [ ]:
np.unique(predict)

### Смотрим метрики

In [ ]:
print(score(gt.flatten(), predict.flatten()))